# Content with notebooks

You can also create content with Jupyter Notebooks. This means that you can include
code blocks and their outputs in your book.

## Markdown + notebooks

As it is markdown, you can embed images, HTML, etc into your posts!

![](https://myst-parser.readthedocs.io/en/latest/_static/logo-wide.svg)

You can also $add_{math}$ and

$$
math^{blocks}
$$

or

$$
\begin{aligned}
\mbox{mean} la_{tex} \\ \\
math blocks
\end{aligned}
$$

But make sure you \$Escape \$your \$dollar signs \$you want to keep!

## MyST markdown

MyST markdown works in Jupyter Notebooks as well. For more information about MyST markdown, check
out [the MyST guide in Jupyter Book](https://jupyterbook.org/content/myst.html),
or see [the MyST markdown documentation](https://myst-parser.readthedocs.io/en/latest/).

## Code blocks and outputs

Jupyter Book will also embed your code blocks and output in your book.
For example, here's some sample Matplotlib code:

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)

In [1]:
import tensorflow as tf

# Verificar soporte CUDA/GPU
has_gpu_support = tf.test.is_built_with_cuda()

# Listar GPUs y CPUs
gpu_list = tf.config.list_physical_devices('GPU')
cpu_list = tf.config.list_physical_devices('CPU')

print("TensorFlow compiled with CUDA/GPU support:", has_gpu_support)
print("TensorFlow can access", len(gpu_list), "GPU(s)")
print("Accessible GPUs are:", gpu_list)
print("TensorFlow can access", len(cpu_list), "CPU(s)")
print("Accessible CPUs are:", cpu_list)

TensorFlow compiled with CUDA/GPU support: False
TensorFlow can access 0 GPU(s)
Accessible GPUs are: []
TensorFlow can access 1 CPU(s)
Accessible CPUs are: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("austinreese/craigslist-carstrucks-data")

print("Path to dataset files:", path)

C:\Users\Laura\miniconda3\envs\ml_cpuenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Laura\.cache\kagglehub\datasets\austinreese\craigslist-carstrucks-data\versions\10


In [4]:
import os

path = r"C:\Users\Laura\.cache\kagglehub\datasets\austinreese\craigslist-carstrucks-data\versions\10"

for file in os.listdir(path):
    print(file)


vehicles.csv


In [5]:
!pip install pandas

In [6]:
import pandas as pd

df = pd.read_csv(os.path.join(path, "vehicles.csv"))
df.head()


In [9]:
!pip install scikit-learn

Preprocesamiento

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Definir columnas
num_cols = selector(dtype_include='number')(df)
cat_cols = selector(dtype_include='object')(df)

# Pipelines por tipo
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# ColumnTransformer une los dos pipelines
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, num_cols),
    ('cat', categorical_pipeline, cat_cols)
])


Creación de la variable binaria HighDemand

In [19]:
# Crear variable binaria 'HighDemand' basada en la mediana del precio
df['HighDemand'] = (df['price'] > df['price'].median()).astype(int)

# Verificar la proporción de clases
print("Distribución de la variable HighDemand:")
print(df['HighDemand'].value_counts(normalize=True))


Distribución de la variable HighDemand:
HighDemand
0    0.500234
1    0.499766
Name: proportion, dtype: float64
